<h1>Recreating Results of Syracuse Paper</h1>

In [1]:
import sys  
sys.path.insert(1, '../src')

In [2]:
from utils import * 

In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("../data/transformed/watermain_breaks_train.csv")
val = pd.read_csv("../data/transformed/watermain_breaks_validation.csv")
test = pd.read_csv("../data/transformed/watermain_breaks_test.csv")

<h2>Feature Engineering</h2>

2015-2018 is our target period

Outcome: if the pipe will breaks in the next 3 years

<h3>Format Dates</h3>

In [5]:
#looking from 2015 trying to predict next 3 years
CUTOFF = pd.to_datetime('01-01-2016')

In [6]:
df['first_break'] = pd.to_datetime(df['first_break'])
df['most_recent_break'] = pd.to_datetime(df['most_recent_break'])
df['INSTALLDAT'] = pd.to_datetime(df['INSTALLDAT'])

In [7]:
df['all_breaks'] = df['all_breaks'].astype(str).apply(lambda s: s.split(","))
df['all_breaks'] = df['all_breaks'].apply(lambda s: [pd.to_datetime(t) for t in s])

Remove any information from after the prediction cutoff

In [8]:
df['breaks_before_cutoff'] = df['all_breaks'].apply(lambda s: [t for t in s if t <= CUTOFF])
df['breaks_after_cutoff'] = df['all_breaks'].apply(lambda s: [t for t in s if t > CUTOFF])

In [9]:
df['first_break'] = df['first_break'].apply(lambda s: np.where(s <= CUTOFF, s, pd.NaT))
df['most_recent_break'] = df['most_recent_break'].apply(lambda s: np.where(s <= CUTOFF, s, pd.NaT))

In [10]:
df['will_break'] = (df['breaks_after_cutoff'].apply(len) > 0).astype(int)

In [11]:
df['will_break'].mean()

0.006550558877047049

<h3>Add Features & Transform Categoricals</h3>

In [12]:
cols = ['PressureSy', 'STATUS', 'MATERIAL', 'SUBTYPE']
df = svm_data_transform_pipeline(df, cols, False)

/Users/haleyjohnson/Desktop/SI_670/watermain_break_predictions/models/../src/utils.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['installation_year'] = df['installation_year'].astype(int)
/Users/haleyjohnson/Desktop/SI_670/watermain_break_predictions/models/../src/utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_installation_to_first_break'] = df.apply(lambda s: break_timedelta(s, 'first_break'), axis = 1)
/Users/haleyjohnson/Desktop/SI_670/watermain_break_predictions/models/..

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17350 entries, 0 to 19234
Data columns (total 46 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   ENABLED                                  17350 non-null  int64         
 1   FACILITYID                               17350 non-null  object        
 2   LOCATION                                 10763 non-null  object        
 3   INSTALLDAT                               17350 non-null  datetime64[ns]
 4   SUBTYPE                                  17350 non-null  object        
 5   MATERIAL                                 17350 non-null  object        
 6   LENGTH                                   17350 non-null  float64       
 7   DIAMETER                                 17350 non-null  float64       
 8   STATUS                                   17350 non-null  object        
 9   PressureSy                               173

In [14]:
df['SUBTYPE'].value_counts()

SUBTYPE
Distribution Main    13612
Hydrant Lead          2350
Transmission Main     1378
Other                    8
Raw Water                2
Name: count, dtype: int64

<h3>Drop Extra Columns</h3>

In [15]:
# to_drop = ['ENABLED', 'FACILITYID', 'LOCATION', 'INSTALLDAT', 'SUBTYPE', 'MATERIAL', 'STATUS', 'PressureSy']
# df = df.drop(columns = to_drop)